In [1]:
from pathlib import Path
import numpy as np
from funcionesTP import *

path_dataset_train = '../TP/template-alumnos/dataset/cats_and_dogs/train'
path_dataset_val = '../TP/template-alumnos/dataset/cats_and_dogs/val'

def cargarDataset(carpeta: Path):
    """
    carpeta Path a la carpeta de train / val
    retorna la matriz de embeddings (1536x2000 / 1536x1000) y la matriz Y (2x2000 / 2x1000)
    """
    
    pathCats = carpeta.joinpath('cats/efficientnet_b3_embeddings.npy')
    pathDogs = carpeta.joinpath('dogs/efficientnet_b3_embeddings.npy')

    embeddingsCats = np.load(pathCats)
    embeddingsDogs = np.load(pathDogs)

    embeddings = np.concatenate((embeddingsCats, embeddingsDogs), axis=1)
    _, m = embeddings.shape

    Y = np.zeros((2,m))
    for i in range(0, int(m/2)):
        Y[0,i] = 1

    for i in range(int(m/2), m):
        Y[1,i] = 1

    return embeddings, Y

def cargarDatasetCompleto():
    """
    retorna el dataset de entrenamiento (X_t, Y_t) y el de valuación (X_v, Y_v)
    """

    
    Xt, Yt = cargarDataset(Path(path_dataset_train))
    Xv, Yv = cargarDataset(Path(path_dataset_val))

    return Xt, Yt, Xv, Yv

Las siguientes funciones generan la matriz W a partir del calculo de la pseudoinversa de X utilizando diferentes metodos de decomposicion

In [2]:
def generar_W_EcuacionesNormales():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    W = pinvEcuacionesNormales(Xt, Yt)
    np.save('./W_eqnormales', W)

def generar_W_QRHH():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    Q, R = QR_con_HH(traspuesta(Xt))
    W = qrFCN(Q, R, Yt)
    np.save('./W_qrhh', W)

def generar_W_QRGS():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    Q, R = QR_con_GS(traspuesta(Xt)) # type: ignore
    W = qrFCN(Q, R, Yt)
    np.save('./W_qrgs', W)

def generar_W_SVD():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    W = svdFCN(Xt, Yt)
    np.save('./W_SVD', W)
    

Utilizando estas cuatro funciones, generamos la matriz de pesos *W* con los distintos métodos

In [ ]:
generar_W_EcuacionesNormales()

In [ ]:
generar_W_QRGS()

In [ ]:
generar_W_QRHH()

In [ ]:
generar_W_SVD()

Creamos la función `test_W()` que nos va a ayudar a testear los métodos midiendo $||Y - WX||_2$ 

In [3]:
def test_W(path_W, dataset = ['val', 'train']):
    """
    path_W Path al archivo de la matriz de pesos W generada con alguno de los métodos
    dataset las matrices que se van a usar para el test. Puede elegir entre entrenamiento o validación     

    utilizando la norma 2, calculamos la norma del residuo ||Y - WX||_2 para tener una medida error
    sobre el conjunto de validación.

    Retorna Norma 2 de la diferencia entre Y y W @ X
    """
    W = np.load(path_W)
    Xv, Yv = cargarDataset(Path(f'../TP/template-alumnos/dataset/cats_and_dogs/{dataset}'))

    matrizDiferencia = Yv - (W @ Xv)
    return np.linalg.norm(matrizDiferencia, ord=2)

Con la ayuda de esta función, midamos la precisión de cada uno de los métodos

In [4]:
metodos = ['./W_qrgs.npy', './W_qrhh.npy', './W_eqnormales.npy', './W_SVD.npy']
for metodo in metodos:
    print('-', metodo)
    print('norma 2 con matrices de entrenamiento: ', test_W(metodo, 'train'))
    print('norma 2 con matrices de validación:    ', test_W(metodo, 'val'))
    print()

- ./W_qrgs.npy
norma 2 con matrices de entrenamiento:  12.70724989849409
norma 2 con matrices de validación:     36.730114624119295

- ./W_qrhh.npy
norma 2 con matrices de entrenamiento:  12.707249898494087
norma 2 con matrices de validación:     36.730114624119246

- ./W_eqnormales.npy
norma 2 con matrices de entrenamiento:  20.60879785919579
norma 2 con matrices de validación:     36.71294057079031

- ./W_SVD.npy
norma 2 con matrices de entrenamiento:  12.707249898494558
norma 2 con matrices de validación:     36.73011483048146



La función `valoresDeConfusion` genera los valores positivos y negativos para cada tipo de clasificación (perros y gatos)

In [5]:
def valoresDeConfusion(path_W):
    """
    path_W Path al archivo de la matriz de pesos W generada con alguno de los métodos
    
    Evalúa el modelo contando cuántos gatos y perros fueron clasificados correctamente y cuántos incorrectamente.

    retorna los verdaderos y falsos positivos de gatos y de perros
    """


    W = np.load(path_W)
    Xv, _ = cargarDataset(Path('../TP/template-alumnos/dataset/cats_and_dogs/val'))
    matriz_resultado = W@Xv 
    gatos_positivos = 0

    for i in range(500):
        resultado = matriz_resultado[:,i]
        if resultado[0] > resultado[1]:
            gatos_positivos += 1 
    
    perros_positivos = 0

    for i in range(500, 1000):
        resultado = matriz_resultado[:,i]
        if resultado[1] > resultado[0]:
            perros_positivos += 1

    return gatos_positivos, 500 - gatos_positivos, perros_positivos, 500 - perros_positivos          


Ahora calculamos los valores que vamos a usar para la matriz de confusión

In [6]:
for metodo in metodos:
    print('-', metodo)
    gatopos, gatoneg, perropos, perroneg = valoresDeConfusion(metodo)
    print('verdaderos positivos gatos:  ',gatopos, 'falsos positivos gatos:      ', gatoneg )
    print(' falsos positivos perros:    ', perroneg, 'verdaderos positivos perros: ',perropos )
    print()


- ./W_qrgs.npy
verdaderos positivos gatos:   334 falsos positivos gatos:       166
 falsos positivos perros:     150 verdaderos positivos perros:  350

- ./W_qrhh.npy
verdaderos positivos gatos:   334 falsos positivos gatos:       166
 falsos positivos perros:     150 verdaderos positivos perros:  350

- ./W_eqnormales.npy
verdaderos positivos gatos:   340 falsos positivos gatos:       160
 falsos positivos perros:     207 verdaderos positivos perros:  293

- ./W_SVD.npy
verdaderos positivos gatos:   334 falsos positivos gatos:       166
 falsos positivos perros:     150 verdaderos positivos perros:  350



para darle soporte empírico al informe, veamos el número de condición de las matrices $X$ y $X^TX$, recordando que éste último es $cond(X)^2$

In [ ]:
Xt, _ = cargarDataset(Path(path_dataset_train))
Xv, _ = cargarDataset(Path(path_dataset_val))

cond_number = np.linalg.cond(Xt)
print(f"Condición de X: {cond_number}")
print(f"Condición de X^T X: {cond_number**2}")

cond_number = np.linalg.cond(Xv)
print(f"Condición de X: {cond_number}")
print(f"Condición de X^T X: {cond_number**2}")

Condición de X: 363.2504577636719
Condición de X^T X: 131950.890625
Condición de X: 195.83343505859375
Condición de X^T X: 38350.734375
